In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
%cd drive/MyDrive/'Colab Notebooks/MetaLearning'
#%cd meta-learning-course-notebooks/1_MAML/
!ls
!pip install import_ipynb --quiet
!pip install learn2learn --quiet
import torch
import numpy as np
import learn2learn as l2l
from learn2learn.data import *
import import_ipynb
import utils

/content/drive/MyDrive/Colab Notebooks/MetaLearning
l2lutils.ipynb	models.ipynb  nb1.ipynb  nb2-CNP.ipynb	nb3.ipynb  utils.ipynb
importing Jupyter notebook from utils.ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class KShotLoader():
    def __init__(self,myds,num_tasks=1000,shots=2,ways=2,classes=None):
        self.shots = shots
        self.ways = ways
        self.myMds = l2l.data.MetaDataset(myds)
        if classes == None:
            n_classes = len(set(myds.labels))
            classes = [i for i in range(n_classes)]
        self.my_tasks = l2l.data.TaskDataset(self.myMds, task_transforms=[
                                l2l.data.transforms.FilterLabels(self.myMds,classes),
                                l2l.data.transforms.NWays(self.myMds,ways),
                                l2l.data.transforms.KShots(self.myMds,2*shots),
                                l2l.data.transforms.LoadData(self.myMds),
                                l2l.data.transforms.RemapLabels(self.myMds),
                                l2l.data.transforms.ConsecutiveLabels(self.myMds)
                                ],num_tasks=num_tasks)
    def get_task(self):
        data,labels = self.my_tasks.sample()
        adaptation_indices = np.zeros(data.size(0), dtype=bool)
        adaptation_indices[np.arange(self.shots*self.ways) * 2] = True
        evaluation_indices = torch.from_numpy(~adaptation_indices)
        adaptation_indices = torch.from_numpy(adaptation_indices)
        adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
        evaluation_data, evaluation_labels = data[evaluation_indices], labels[evaluation_indices]
        d_train = (adaptation_data,adaptation_labels)
        d_test = (evaluation_data,evaluation_labels)
        return d_train, d_test

In [ ]:
meta_train_ds, meta_test_ds, _ = utils.euclideanDataset(n_samples=100,n_features=20,n_classes=10,batch_size=32)

In [ ]:
classes_train = [i for i in range(5)]
classes_test = [i+5 for i in range(5)]
classes_train, classes_test

([0, 1, 2, 3, 4], [5, 6, 7, 8, 9])

In [ ]:
kloader=KShotLoader(meta_train_ds,ways=10)

In [ ]:
d_train,d_test=kloader.get_task()

In [ ]:
d_train[1]

tensor([8, 8, 5, 5, 3, 3, 0, 0, 2, 2, 6, 6, 9, 9, 1, 1, 4, 4, 7, 7])

In [ ]:
d_train,d_test=kloader.get_task()

In [ ]:
d_train

(tensor([[  1.4252,  -5.3629,  -7.4316,  -1.2353,   4.1250,  -3.4136,   5.4762,
            6.8715,  -2.1126,   2.8870,  -5.5260,   3.5394,  -2.7973,  -1.6151,
            0.0789,  -2.8429,  -1.1390,  -2.4192,  -0.2087,  -2.0342],
         [  0.6569,  -3.9452,  -0.9884,  -4.2529,   7.9883,  -0.6208,  -1.4668,
            0.4669,   3.6002,   2.1912,  -4.4996,   0.2898,  -4.6931,  -5.7488,
            4.2079,  -1.6429,  -3.1174,  -1.6917,   0.5756,  -0.7232],
         [ -0.1299,   0.2805,  -0.4811,   4.3883,   3.2874,   1.4188,  -3.9281,
            5.6626,   2.2771,  -7.6975,   6.0267,  -1.6949,   4.0686,   3.6706,
            0.0118,  -1.3412,   5.5820,  -2.7030,  -1.3392,   3.0803],
         [ -0.2881,   3.1426,  -1.2103,  -2.4867,   4.0456,  -1.2104,  -0.9982,
            0.0962,   3.5991,   0.2794,   8.1636,  -0.8992,   0.0339,  -4.0489,
           -2.5951,  -0.8657,   2.8465,   3.9900,   0.8443,   7.3409],
         [ -3.8342,  -0.5827,   3.7769,   6.9896,   4.7253,   2.0283,  -2.91